augmentations 1

how to use this notebook

1. mount drive
2. install raserio
3. run script
4. check new folder
5. confirm directory
link to data [HOTOSM Datasets](https://drive.google.com/drive/folders/1jK4ua8xNxsEB4ArK6GbmsnPoEj3KBrJU?usp=drive_link)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 50.7 MB/s eta 0:00:00


In [14]:
import albumentations as A
import rasterio
import os

# Define your augmentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.GaussNoise(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
])

# Specify the directory containing the images
image_dir = '/content/drive/MyDrive/HOTSM_madakixo/augmented_data/Task0201_Augmentation/training_399 (1)/preprocessed/chips'
output_base_dir = '/content/drive/MyDrive/HOTSM_madakixo/augmented_data/training_385'

# Create directories for the augmented images
os.makedirs(os.path.join(output_base_dir, 'flipped'), exist_ok=True)
os.makedirs(os.path.join(output_base_dir, 'rotated'), exist_ok=True)
os.makedirs(os.path.join(output_base_dir, 'noised'), exist_ok=True)

# Loop over the images in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.tif'):
        with rasterio.open(os.path.join(image_dir, filename)) as src:
            image = src.read()

        # Apply transformations
        flipped_h = transform(image=image)['image']
        flipped_v = transform(image=image)['image']
        rotated = transform(image=image)['image']
        noised = transform(image=image)['image']

        # Save the augmented images (rasterio)
        with rasterio.open(
            os.path.join(output_base_dir, 'flipped', f'{filename[:-4]}_flipped_h.tif'),
            'w',
            **src.meta  # Retain metadata
        ) as dst:
            dst.write(flipped_h)

        with rasterio.open(
            os.path.join(output_base_dir, 'flipped', f'{filename[:-4]}_flipped_v.tif'),
            'w',
            **src.meta
        ) as dst:
            dst.write(flipped_v)

        with rasterio.open(
            os.path.join(output_base_dir, 'rotated', f'{filename[:-4]}_rotated.tif'),
            'w',
            **src.meta
        ) as dst:
            dst.write(rotated)

        with rasterio.open(
            os.path.join(output_base_dir, 'noised', f'{filename[:-4]}_noised.tif'),
            'w',
            **src.meta
        ) as dst:
            dst.write(noised)


In [12]:
!# size of flipped_v, flipped_h

print(f"Flipped horizontally: {flipped_h.shape}")
print(f"Flipped vertically: {flipped_v.shape}")


Flipped horizontally: (3, 256, 256)
Flipped vertically: (3, 256, 256)


In [13]:
# size of noised,  rotated and image_dir

print(f"Noised image shape: {noised.shape}")
print(f"Rotated image shape: {rotated.shape}")
print(f"Image directory path: {image_dir}")


Noised image shape: (3, 256, 256)
Rotated image shape: (3, 256, 256)
Image directory path: /content/drive/MyDrive/HOTSM_madakixo/augmented_data/Task0201_Augmentation/training_399 (1)/preprocessed/chips


In [ ]:
import cv2

#...Inside your augmentation loop ...
def add_fog(image, intensity=0.3):
    overlay = image.copy()
    overlay[:] = [200, 200, 200]  # Grayish fog color
    foggy_image = cv2.addWeighted(overlay, intensity, image, 1 - intensity, 0)
    return foggy_image

# ... Apply fog
foggy = add_fog(image, intensity=0.4)  # Adjust intensity
